In [ ]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm
import time
import os

# Load the full dataset
df = pd.read_csv('data_final/cn_news.csv')

# Divide into 10 batches
n_batches = 10
batch_size = len(df) // n_batches

# Initialize translator
translator = Translator()
tqdm.pandas()

def safe_translate(text):
    try:
        return translator.translate(text, src='zh-cn', dest='en').text
    except Exception as e:
        print(f"⚠️ Translation error: {e}")
        return "[Translation failed]"

# Output folder to store temporary results
output_dir = "cn_batches"
os.makedirs(output_dir, exist_ok=True)

for i in range(n_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size if i < n_batches - 1 else len(df)
    batch = df.iloc[start_idx:end_idx]
    
    print(f"\n🚀 Translating batch {i+1}/{n_batches} ({start_idx} to {end_idx})")

    translated = batch['Headlines'].progress_apply(safe_translate)
    translated_df = pd.DataFrame({'Headlines_English': translated})

    batch_output_path = os.path.join(output_dir, f'cn_news_translated_batch_{i+1}.csv')
    translated_df.to_csv(batch_output_path, index=False)

    print(f"✅ Saved batch {i+1} to {batch_output_path}")
    time.sleep(3)



🚀 Translating batch 1/10 (0 to 357)


100%|██████████| 357/357 [03:51<00:00,  1.54it/s]


✅ Saved batch 1 to data_final/batches/cn_news_translated_batch_1.csv

🚀 Translating batch 2/10 (357 to 714)


100%|██████████| 357/357 [04:21<00:00,  1.36it/s]


✅ Saved batch 2 to data_final/batches/cn_news_translated_batch_2.csv

🚀 Translating batch 3/10 (714 to 1071)


...

✅ Saved batch 9 to data_final/batches/cn_news_translated_batch_9.csv

🚀 Translating batch 10/10 (3213 to 3578)


 83%|████████▎ | 303/365 [06:10<02:22,  2.29s/it]

⚠️ Translation error: The read operation timed out


100%|██████████| 365/365 [07:25<00:00,  1.22s/it]


✅ Saved batch 10 to data_final/batches/cn_news_translated_batch_10.csv


In [3]:
import glob
import re

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('(\d+)', s)]

input_files = sorted(glob.glob('cn_batches/cn_news_translated_batch_*.csv'), key=natural_sort_key)

for file in input_files:
    try:
        df = pd.read_csv(file, encoding='utf-8')
        df.to_csv(file, index=False, encoding='utf-8-sig')
    except Exception as e:
        print(f"❌ Error with file {file}: {e}")

df_original = pd.read_csv('data_final/cn_news.csv')
df_translated = pd.concat([pd.read_csv(f, encoding='utf-8-sig') for f in input_files], ignore_index=True)

df_final = df_original[['time']].copy()
df_final['Headlines'] = df_translated['Headlines_English']

final_output_path = 'data_final/cn_news_to_en.csv'
df_final.to_csv(final_output_path, index=False, encoding='utf-8-sig')

print("✅ Merged batches saved in correct order to:", final_output_path)


✅ Merged batches saved in correct order to: data_final/cn_news_to_en.csv
